In [39]:
import psycopg2
import random
import requests
import numpy as np
from numpy.random import choice

In [50]:
conn = psycopg2.connect(dbname='admt_g02', user='admt_g02', password='xi6aeSoo', host='alcor.inf.unibz.it', port=5433)
cur = conn.cursor()

In [51]:
cur.execute('Delete from warehouse.operator')
cur.execute('Delete from warehouse.machine')

In [52]:
res = requests.get(f'http://uinames.com/api/?region=austria&amount=100')
operators = res.json()

values = []
for operator in operators:
    values.append(f'(\'{operator["name"]} {operator["surname"]}\')')
    
query_args = ','.join(values)
cur.execute(f'insert into warehouse.operator (operator_name) values {query_args}')

In [53]:
res = requests.get(f'http://uinames.com/api/?amount=250')
machines = [m['surname'] for m in res.json()]

res = requests.get(f'http://uinames.com/api/?amount=20')
vendors = [v['surname'] for v in res.json()]

years = list(range(1995, 2011))

values = []
for machine in machines:
    
    p = np.linspace(20,5,len(vendors))
    p = p / np.sum(p)
    vendor = choice(vendors, p=p)
    
    year = choice(years)   
    values.append(f'(\'{machine}\', \'{vendor}\', {year})')
    
query_args = ','.join(values)
cur.execute(f'insert into warehouse.machine (machine_name, machine_vendor, machine_purchasing_year) values {query_args}')

In [54]:
conn.commit()
cur.close()
conn.close()